### Collborative Filtering Model 
Student Name : Sunil Kumar J S <br>
Analytical Goal : Recommend customized songs for user based on user <br> 
Machine Learning Algorithm : Alternatie Least Square ( Variant of Matrix Factorization)

In [0]:
from pyspark.sql import SparkSession


from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
#from pyspark.ml.feature import OneHotEncoder
#from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
#from pyspark.ml.classification import LogisticRegression
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer

In [0]:
spark = SparkSession.builder.config('spark.sql.parquet.binaryAsString', 'true') \
            .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
            .appName("OMS_lyrics") \
            .getOrCreate()

In [0]:
aws_access_key = 'ACCESS KEY'
aws_secret_key = 'SECRET KEY'
spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1')
#spark._jsc.hadoopConfiguration().set('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1') 
spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
#spark._jsc.hadoopConfiguration().set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_key)

In [0]:
spark._jsc.hadoopConfiguration().set('spark.network.timeout','7200s')
spark._jsc.hadoopConfiguration().set('spark.executor.heartbeatInterval','1200s')

In [0]:
spark = SparkSession.builder.config('spark.sql.parquet.binaryAsString', 'true') \
            .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1')\
            .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1')\
            .config("spark.jars.packages", "ai.h2o:sparkling-water-package_2.12:3.36.0.3-1-3.2")\
            .config("spark.network.timeout", "36000000s")\
            .config("spark.executor.heartbeatInterval", "7200s")\
            .config("spark.ext.h2o.fail.on.unsupported.spark.param", "false")\
            .config("spark.dynamicAllocation.enabled", "false")\
            .appName("OMS") \
            .getOrCreate()

In [0]:
database = 'oms'
collection = 'play_counts'
user_name = 'user'
password = 'user'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
triplets_df = spark.read.format("mongo").option("uri",connection_string).load()

### Cross Validation on 5% of the data

In [0]:
triplets_df2 =triplets_df.sample(0.05).cache()

In [0]:
triplets_df2.count()

Out[89]: 2420757

##### Encoding of user-id and song-id

In [0]:
def indexStringColumns(df, cols):
    newdf = df
    
    for c in cols:
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        newdf = sm.transform(newdf)
    return newdf
triplets_encoded = indexStringColumns(triplets_df2, ["user_id", "song_id"])

In [0]:
als = ALS(
         userCol="user_id-num", 
         itemCol="song_id-num",
         ratingCol="play_count", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop"
)

In [0]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10]) \
            .addGrid(als.regParam, [1,.01,0.001]) \
            .build()
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="play_count", 
           predictionCol="prediction") 
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [0]:
splits=triplets_encoded.randomSplit([0.8,0.2])
train = splits[0].cache()
test =splits[0].cache()
model = cv.fit(train)
best_model = model.bestModel

In [0]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

1.7795351470292253

In [0]:
best_model._java_obj.parent().getRegParam()

#### Modelling on Full data with best hyper-parameter selected from cross validation using 5% of the data

In [0]:
triplets_full =triplets_df.cache()

In [0]:
triplets_full.count()

Out[11]: 48373586

In [0]:
triplets_encoded = indexStringColumns(triplets_full, ["user_id", "song_id"])

In [0]:
database = 'oms'
collection = 'play_counts'
user_name = 'oms'
password = 'oms'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
triplets_encoded.write.format("mongo").option("uri",connection_string).mode("overwrite").save()

In [0]:
als = ALS(
         userCol="user_id-num", 
         itemCol="song_id-num",
         ratingCol="play_count", 
         nonnegative = True, 
         implicitPrefs = False,
         coldStartStrategy="drop",
         regParam=1
)

In [0]:
splits=triplets_encoded.randomSplit([0.8,0.2])
train = splits[0].cache()
test =splits[0].cache()
model = als.fit(train)
#Extract best model from the cv model above
test_predictions = model.transform(test)

In [0]:
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

4.273457273357628

## Writing user embeddings,item embeddings and recommendations in mongodb

In [0]:
user_mapping = train.drop_duplicates(subset=['user_id']).select('user_id','user_id-num')
user_df=model.userFactors.drop_duplicates(subset=['id'])\
.withColumnRenamed('id','user_id-num').join(user_mapping,on='user_id-num')

In [0]:
items_mapping = train.drop_duplicates(subset=['song_id']).select('song_id','song_id-num')
item_df=model.itemFactors.drop_duplicates(subset=['id'])\
.withColumnRenamed('id','song_id-num').join(items_mapping,on='song_id-num')

In [0]:
database = 'oms'
collection = 'user-emb'
user_name = 'oms'
password = 'oms'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
user_df.write.format("mongo").option("uri",connection_string).mode("overwrite").save()

In [0]:
user_df.display()

user_id-num,features,user_id
0,"List(0.15358827, 0.16901688, 0.16681913, 0.11970834, 0.12270605, 0.14261079, 0.16516069, 0.16668352, 0.16577068, 0.1816415)",ec6dfcf19485cb011e0b22637075037aae34cf26
8,"List(1.0456876, 0.66107255, 1.173021, 0.5986011, 0.36869505, 1.7434124, 0.89344454, 4.106286, 0.4695863, 0.8567737)",119b7c88d58d0c6eb051365c103da5caf817bea6
67,"List(0.28559512, 0.3236838, 0.31125364, 0.25884816, 0.27121317, 0.46362615, 0.34182864, 0.39745253, 0.31654292, 0.48651797)",a4ccc36714975978b545e35db83584fa9f7fa6c6
69,"List(0.75713354, 0.555435, 0.68607354, 0.24718912, 0.4933887, 0.6436879, 1.0328714, 0.5066358, 0.73272383, 0.9000142)",e04a9e4861256079476c25d655d97c1c5603b3cd
70,"List(0.34557104, 0.24918419, 0.21692355, 0.2519352, 0.23906022, 0.3138424, 0.30904168, 0.31422567, 0.30974913, 0.30528235)",062eef2a03b53d2b10f5018135e3361659c6a3bf
147,"List(0.31776512, 0.31570396, 0.3316018, 0.22171403, 0.22493662, 0.329631, 0.35015583, 0.3184566, 0.36854059, 0.36492053)",34516b707f840a994a5c7876a6430d2a3b446eb4
160,"List(0.32798877, 0.28719494, 0.30941644, 0.25214675, 0.2432099, 0.31909457, 0.1804114, 0.25886112, 0.33532226, 0.2331445)",bd94799ca4b7e5fadbeecf51637d522a17a0659d
168,"List(0.18594053, 0.2047123, 0.2159944, 0.17506745, 0.17925015, 0.2658221, 0.2119272, 0.21026936, 0.21090724, 0.2626017)",a771391d088269c5dc7adcc9e7e0e77d3cf8b80b
169,"List(0.2915029, 0.3236473, 0.54600894, 0.2445144, 0.18272246, 0.26639894, 0.34131625, 0.25263977, 0.4673221, 0.34359822)",4ee2a82d9d9f5376f0a6625455fd040b1a5ba813
170,"List(0.27415568, 0.22061719, 0.20466985, 0.204874, 0.22145632, 0.31485432, 0.3121141, 0.26331258, 0.28976437, 0.36925644)",237707321971d7f90362cdb41ed355fc8592e7ce


In [0]:
user_df.printSchema()

root
-- user_id-num: integer (nullable = false)
-- features: array (nullable = true)
 |-- element: float (containsNull = false)
-- user_id: string (nullable = false)

In [0]:
database = 'oms'
collection = 'item-emb'
user_name = 'oms'
password = 'oms'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
item_df.write.format("mongo").option("uri",connection_string).mode("overwrite").save()

In [0]:
database = 'oms'
collection = 'recommendations'
user_name = 'oms'
password = 'oms'
address = 'oms-cluster.0navm.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"
recommendations = model.recommendForAllUsers(10)
recommendations.write.format("mongo").option("uri",connection_string).mode("overwrite").save()